In [ ]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torch.utils.tensorboard.writer import SummaryWriter
from model import VGG19

step 1 ：准备数据集

In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),  
    transforms.ToTensor(), 
    ])

train_data = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_data = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)


In [ ]:
print(f'训练数据集长度：{len(train_data)}')
print(f'测试数据集长度：{len(test_data)}')
print(f'训练数据集的类别数：{len(train_data.classes)}')

step 2: dataloader 数据加载

In [ ]:
train_data_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_data_loader = DataLoader(test_data, batch_size=64, shuffle=True)

In [ ]:
for i, (images, labels) in enumerate(train_data_loader):
    print(f'第{i}个batch')
    print(f'images.shape: {images.shape}')
    print(f'labels.shape: {labels.shape}')
    break

step 3：模型定义

In [ ]:
model = VGG19()
print(model)

step 4: 定义损失函数

In [ ]:
loss_fn = nn.CrossEntropyLoss()

step 5: 定义优化器

In [ ]:
learning_rate = 0.001
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
writer = SummaryWriter('./logs/vgg19')

step 6: 训练

In [ ]:
epochs = 10
for epoch in range(epochs):
    model.train()
    for i, (images, labels) in enumerate(train_data_loader):
        outputs = model(images)
        loss = loss_fn(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 10 == 0:
            print(f'Epoch [{epoch+1}/{epochs}], Step [{i+1}/{len(train_data_loader)}], Loss: {loss.item():.4f}')
            writer.add_scalar('loss', loss.item(), epoch * len(train_data_loader) + i)

    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in test_data_loader:
            outputs = model(images)
            _, predicted = torch.max(outputs.data, dim=1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        print(f'Accuracy of the model on the test images: {100 * correct / total}%')
        writer.add_scalar('accuracy', 100 * correct / total, epoch)